In [1]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# Create a blank canvas to draw on
canvas = np.zeros((480, 640, 3), np.uint8) + 255  # White canvas

# Drawing parameters
draw_color = (0, 0, 255)  # Red color (BGR format)
thickness = 5
prev_x, prev_y = None, None
drawing = False

def detect_index_finger(frame):
    """Detect index finger tip using MediaPipe"""
    # Convert BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the image and find hands
    result = hands.process(rgb_frame)
    
    # Draw hand landmarks on the frame
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw landmarks
            mp_drawing.draw_landmarks(
                frame, 
                hand_landmarks, 
                mp_hands.HAND_CONNECTIONS
            )
            
            # Get index finger tip coordinates (landmark 8)
            index_finger = hand_landmarks.landmark[8]
            x, y = int(index_finger.x * frame.shape[1]), int(index_finger.y * frame.shape[0])
            
            # Get middle finger tip coordinates (landmark 12) to check gesture
            middle_finger = hand_landmarks.landmark[12]
            middle_x, middle_y = int(middle_finger.x * frame.shape[1]), int(middle_finger.y * frame.shape[0])
            
            # Calculate distance between index and middle finger tips
            distance = ((x - middle_x) ** 2 + (y - middle_y) ** 2) ** 0.5
            
            # Draw circle at index fingertip
            cv2.circle(frame, (x, y), 10, draw_color, -1)
            
            # Return fingertip position and whether we should be drawing
            # Draw only when index finger is up but middle finger is down
            draw_mode = distance > 50
            return (x, y), draw_mode
    
    return None, False

def add_buttons(frame):
    """Add color and action buttons to the frame"""
    # Red button
    frame = cv2.rectangle(frame, (10, 10), (90, 60), (0, 0, 255), -1)
    cv2.putText(frame, "Red", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Green button
    frame = cv2.rectangle(frame, (100, 10), (180, 60), (0, 255, 0), -1)
    cv2.putText(frame, "Green", (110, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Blue button
    frame = cv2.rectangle(frame, (190, 10), (270, 60), (255, 0, 0), -1)
    cv2.putText(frame, "Blue", (210, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Eraser button
    frame = cv2.rectangle(frame, (280, 10), (360, 60), (0, 0, 0), -1)
    cv2.putText(frame, "Erase", (300, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Clear button
    frame = cv2.rectangle(frame, (370, 10), (450, 60), (255, 255, 255), -1)
    cv2.putText(frame, "Clear", (390, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    
    return frame

def check_button_press(x, y, canvas):
    """Check if a button was pressed and take appropriate action"""
    global draw_color
    
    if y < 60:  # Button area
        if 10 <= x <= 90:  # Red button
            draw_color = (0, 0, 255)
            return True, canvas
        elif 100 <= x <= 180:  # Green button
            draw_color = (0, 255, 0)
            return True, canvas
        elif 190 <= x <= 270:  # Blue button
            draw_color = (255, 0, 0)
            return True, canvas
        elif 280 <= x <= 360:  # Eraser
            draw_color = (255, 255, 255)  # White for eraser
            return True, canvas
        elif 370 <= x <= 450:  # Clear all
            canvas = np.zeros((480, 640, 3), np.uint8) + 255  # Reset to white
            return True, canvas
    
    return False, canvas

def display_images(frame, canvas):
    """Display both the webcam feed and canvas side by side"""
    plt.figure(figsize=(12, 5))
    
    plt.subplot(121)
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title('Webcam Feed')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
    plt.title('Drawing Canvas')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

def run_drawing_app(frames=300):
    """Run the drawing app for a specified number of frames"""
    global canvas, prev_x, prev_y
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Cannot open webcam")
        return canvas
    
    # Set resolution for better performance
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    # Instructions
    print("Touchless Drawing Canvas")
    print("------------------------")
    print("1. Point with your index finger to draw")
    print("2. Separate index and middle fingers to start drawing")
    print("3. Keep fingers together to stop drawing")
    print("4. Move to the colored buttons at top to change colors")
    print("5. Move to 'Clear' to reset canvas")
    
    for i in range(frames):
        # Read frame from webcam
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break
            
        # Flip the frame horizontally for a more intuitive mirror view
        frame = cv2.flip(frame, 1)
        
        # Add buttons to the frame
        frame = add_buttons(frame)
        
        # Detect index finger position
        finger_position, should_draw = detect_index_finger(frame)
        
        if finger_position:
            x, y = finger_position
            
            # Check if a button was pressed
            button_pressed, canvas = check_button_press(x, y, canvas)
            
            if not button_pressed and should_draw:
                # Draw on canvas if we should be drawing
                if prev_x is not None and prev_y is not None:
                    cv2.line(canvas, (prev_x, prev_y), (x, y), draw_color, thickness)
                prev_x, prev_y = x, y
            elif not should_draw:
                # Reset previous position if not drawing
                prev_x, prev_y = None, None
        else:
            # Reset previous position if finger not detected
            prev_x, prev_y = None, None
        
        # Display images (clear previous output)
        if i % 2 == 0:  # Update display every 2 frames for better performance
            clear_output(wait=True)
            display_images(frame, canvas)
    
    # Release webcam
    cap.release()
    
    print("Session ended. Your final drawing is displayed above.")
    return canvas

def save_drawing(canvas, filename="my_drawing.png"):
    """Save the canvas to a file"""
    cv2.imwrite(filename, canvas)
    print(f"Drawing saved as {filename}")

In [2]:
# Run the drawing app for the specified number of frames
final_canvas = run_drawing_app(frames=500)  # About 30 seconds of drawing time

# Save your drawing (optional)
save_drawing(final_canvas, "my_masterpiece.png")

KeyboardInterrupt: 